In [2]:
# Imports

import string
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style
sns.set_style("whitegrid")

# Increase the width of the notebook for displaying DataFrames
display(HTML("<style>.container { width:75% !important; }</style>"))

C:\Users\Madi\AppData\Local\Temp\ipykernel_388\115617131.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### *Loading Dataset*

In [4]:
df = pd.read_csv('data.csv')

C:\Users\Madi\AppData\Local\Temp\ipykernel_388\1507164070.py:1: DtypeWarning: Columns (0,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


### *Adding 'AverageSalary' Category and Displaying the Head of Dataset to Describe its Properties*

In [5]:
df = df.assign(AverageSalary=((df["HighestSalary"]+df["LowestSalary"])/2))
df.head(5)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN,15.0
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN,15.0
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN,15.0


___
# **PART 1 - Data Preparation and Preprocessing**
___
## 1) Describing the dataset

In [6]:
rows, cols = df.shape
unique_counts = df.nunique()
print(unique_counts) # make sure this prints correctly
print(f"{rows} rows , {cols} cols in dataframe.")

Id                   318477
Title                168065
Company               40628
Date                    163
Location                 65
Area                     19
Classification           30
SubClassification       338
Requirement          234287
FullDescription      250901
LowestSalary             11
HighestSalary            11
JobType                   4
AverageSalary            11
dtype: int64
318477 rows , 14 cols in dataframe.


The dataset structure is such that each row represents an individual job listing. There are 14 possible attributes a listing can have, and it is possible that not every one is filled. The original 13 categories are: Id, Title, Company, Date, Location, Area, Classification, SubClassification, Requirement, FullDescription, LowestSalary and HighestSalary. AverageSalary was added as a 14th category using ```df.assign``` and will be treated the same as each other feature.

* Id is a category for which each listing has a unique Id. It is used in some calculations, but is not particularly descriptive.

* Company and Title represent the company a listing is being advertised for and the job title the listing is associated with. There are 40,628 companies listing jobs and 168,065 unique job titles advertsied.

* Classification and SubClassification are two categories representing the sector and sub-sector a job belongs to. For example, the Classification may be 'Information & Communication Technology' and the SubClassification may be 'Developers/Programmers'.

* Requirement and FullDescription are two descriptive categories with a high degree of uniqueness. They appear to lack a standard format, and it is like that text based data analytics techniques will need to be leveraged to gain insights from these features.

* HighestSalary, LowestSalary and AverageSalary are salary metrics which will be used to explore relationships between location, sector, and salaries offered.

* Location and Area classify where a listing is based. There are many locations representing the city but only 19 unique areas which specify where in a city a listing is advertised for, indicating a potentially large amount of missing data. Location in particular will be used extensively in the job metadata section.

At this stage it seems that all of the attributes will contribute in some way to the analysis, so they will not be removed during the pre-processing stage.

It is difficult to classify if a listing is invalid so this analysis will operate under the assumption that all listings are valid and nothing will be removed. 

___
## 2) Describing the Steps Used for Data Preparation and Preprocessing

In [7]:
##Hint replace(to_replace=r'&.*', value='', regex=True)
pattern = r"&.*"
df["Id"] = df["Id"].replace(to_replace=pattern, value='', regex=True)

pattern = r"T(.*)"
df["Date"] = df["Date"].replace(to_replace=pattern, value='', regex=True)

df.dtypes

Id                    object
Title                 object
Company               object
Date                  object
Location              object
Area                  object
Classification        object
SubClassification     object
Requirement           object
FullDescription       object
LowestSalary           int64
HighestSalary          int64
JobType               object
AverageSalary        float64
dtype: object

In [8]:
# Set column "Id" to type int64
df["Id"] = df["Id"].astype(np.int64)

# Set column "Date" to type datetime64[ns]
df["Date"] = pd.to_datetime(df["Date"]) 
# Data types after data cleaning and conversions
df.dtypes

Id                            int64
Title                        object
Company                      object
Date                 datetime64[ns]
Location                     object
Area                         object
Classification               object
SubClassification            object
Requirement                  object
FullDescription              object
LowestSalary                  int64
HighestSalary                 int64
JobType                      object
AverageSalary               float64
dtype: object

In [9]:
earliest_date = df['Date'].min()
latest_date = df['Date'].max()

print(f"The data is from {earliest_date} to {latest_date}")

The data is from 2018-10-01 00:00:00 to 2019-03-13 00:00:00


The data was loaded in using ```pd.from_csv('data.csv')``` and using ```df.head(5)```, information about the features of the data was described. 

Before performing any calculations or analysis, the columns must be represented in their proper datatypes. To do so, the numeric columns must be cleaned due to inconsistencies in their formatting. This was done using regex and ```.replace()``` for the Id and Date columns.

After cleaning the columns, the Id column was converted to an integer datatype using ```.astype(int)``` and the Date column was converted to datetime64[ns] type using ```pd.to_datetime(df['Date])```.

As a byproduct of this cleaning, the date range of the data could be extrapolated and is seen above.

The only data normalization performed was calculating the average salary using ```df.assign()``` as described in part 1.1. This was done to have an extra metric to compare the salaries for job as the ranges may not be directly comparable.

In [10]:
#Save the cleaned and preprocessed dataset
df.to_csv("preprocessed_data.csv")

Finally, the preprocessed data was saved in an extra csv to save computational resources for each run of the file.

___
## 3) Hypothesis
This analysis will aim to investigate what factors lead to the most lucrative, flexible and in demand job opportunities in Australia. The factors investigated will be the location and sector of job listings. Various sub-factors will also be considered, such as the sub-sector and job skills of listings.

The team hypothesises that jobs in healthcare and IT will see the most listings with healthcare offering the most variety in viable places to live and IT offering the highest wages relative to the number of job listings. The team also hypothesises that wages and market size will be the highest in capitol cities, and that the markets in these cities will favour skilled workers.